# Preamble

## Drive integration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## GPU

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Free GPU memory

In [ ]:
import gc
def free_gpu_memory():
  gc.collect()
  torch.cuda.empty_cache()

# Classifier Main

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import pandas as pd

In [ ]:
q1 = pd.read_csv('/content/drive/MyDrive/data/csv/queries.csv')

In [ ]:
q1.head(5)

In [ ]:
label_mapping = {'quantitative analysis': 0, 'general information': 1, 'miscellaneous':2}
q1['label'] = q1['label'].map(label_mapping)

In [ ]:
# Load the pre-trained BioBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=3)

In [ ]:
free_gpu_memory()